In [29]:
import os
import cv2
import numpy as np
import h5py
import glob
from keras._tf_keras.keras.preprocessing.image import img_to_array
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator
from mtcnn import MTCNN
from keras._tf_keras.keras.models import Sequential
from keras._tf_keras.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
# Load OpenCV's Haar Cascade Face Detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


# Load MTCNN Face Detector
mtcnn_detector = MTCNN()

# Image Size
IMG_SIZE = (200, 200)


# Dataset Paths
folder_path = "D:/fold/Project/age_det/dataset/combined_faces/content/data"

h5_file = "dataset.h5"


In [31]:
def load_and_preprocess(img_path):
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error loading image: {img_path}")
        return None
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, IMG_SIZE)
    img = img.astype("float32") / 255.0
    return img_to_array(img)

In [32]:
def detect_and_crop_face(img_path):
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error: Unable to read {img_path}")
        return None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Try Haar Cascade
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.02, minNeighbors=4, minSize=(30, 30))

    
    if len(faces) == 0:
        # Try MTCNN
        mtcnn_faces = mtcnn_detector.detect_faces(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if len(mtcnn_faces) > 0:
            x, y, w, h = mtcnn_faces[0]['box']
        else:
            print(f"No face detected in {img_path}")
            return None
    else:
        x, y, w, h = faces[0]
    
    # Crop and resize face
    face = img[y:y+h, x:x+w]
    face = cv2.resize(face, IMG_SIZE)
    return face.astype("float32") / 255.0

In [33]:
# Load Images from Folder

# Load Images from Folder
def load_img_folder(folder_path):
    img_data = []
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"Folder not found: {folder_path}")
    
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith((".jpg", ".png", ".jpeg"))]
    if not image_files:
        print(" No images found in dataset folder!")
        return np.array([])
    
    for filename in image_files:
        img_path = os.path.join(folder_path, filename)
        img_array = load_and_preprocess(img_path)
        if img_array is not None:
            img_data.append(img_array)
    
    return np.array(img_data)

In [34]:
if not os.path.exists(h5_file):
    with h5py.File(h5_file, "w") as hf:
        img_data, age_labels = [], []
        
        for img_path in glob.glob(os.path.join(folder_path, "*.jpg")):
            try:
                filename = os.path.basename(img_path)
                age = int(filename.split("_")[0])  # Extract age from filename
                img = detect_and_crop_face(img_path)
                
                if img is not None:
                    img_data.append(img)
                    age_labels.append(age)
            except Exception as e:
                print(f" Skipping {img_path}: {e}")

        if img_data:
            hf.create_dataset("images", data=np.array(img_data, dtype="float32"))
            hf.create_dataset("ages", data=np.array(age_labels, dtype="float32"))
            print(f" Dataset stored in {h5_file}")
        else:
            print(" No valid data to store in dataset!")

In [35]:
# Load Processed Dataset
X_train, y_train = [], []
image_paths = glob.glob(os.path.join(folder_path, "*.jpg"))

for img_path in image_paths:
    try:
        filename = os.path.basename(img_path)
        age = int(filename.split("_")[0])
        img = load_and_preprocess(img_path)
        if img is not None:
            X_train.append(img)
            y_train.append(age)
    except Exception as e:
        print(f" Skipping {img_path}: {e}")

X_train = np.array(X_train, dtype="float32")
y_train = np.array(y_train, dtype="float32")
print(f" Dataset Loaded: {X_train.shape[0]} images")


 Dataset Loaded: 0 images


In [36]:
# Model Architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')  # Regression output for age
])

In [37]:
# Compile and Train Model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 97, 97, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 46, 46, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 23, 23, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 67712)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │    17,334,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,428,033 (66.48 MB)

 Trainable params: 17,428,033 (66.48 MB)

 Non-trainable params: 0 (0.00 B)

In [38]:
model.fit(X_train, y_train, batch_size=32, epochs=20, validation_split=0.2)

ValueError: Training data contains 0 samples, which is not sufficient to split it into a validation and training set as specified by `validation_split=0.2`. Either provide more data, or a different value for the `validation_split` argument.

In [ ]:
# Save Model
model.save("age_detection_model.h5")
print(" Model saved successfully!")

✅ Model saved successfully!
